In [1]:
from virtualreality.dataset import VirtualReality
from virtualreality.utilities import get_block_repetition

import numpy as np
import mne
import moabb
import pandas as pd

from moabb.paradigms import P300
from pyriemann.estimation import ERPCovariances
from pyriemann.classification import MDM

from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import KFold
import joblib

from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")

<frozen importlib._bootstrap>:241: FutureWarning: mne.io.pick.pick_types is deprecated will be removed in 1.6, use documented public API instead. If no appropriate public API exists, please open an issue on GitHub.
<frozen importlib._bootstrap>:241: FutureWarning: mne.io.pick.pick_channels_regexp is deprecated will be removed in 1.6, use documented public API instead. If no appropriate public API exists, please open an issue on GitHub.
<frozen importlib._bootstrap>:241: FutureWarning: mne.io.pick.channel_type is deprecated will be removed in 1.6, use documented public API instead. If no appropriate public API exists, please open an issue on GitHub.


To use the get_shape_from_baseconcar, InputShapeSetterEEG, BraindecodeDatasetLoaderyou need to install `braindecode`.`pip install braindecode` or Please refer to `https://braindecode.org`.


/Users/sid/Documents/GitHub/XR_EEG_pipeline_validation/py.VR.EEG.2018-GIPSA/env_311/lib/python3.11/site-packages/moabb/pipelines/__init__.py:26: ModuleNotFoundError: Tensorflow is not installed. You won't be able to use these MOABB pipelines if you attempt to do so.
  warn(



=============================
Classification of the trials
=============================

This example shows how to extract the epochs from the dataset of a given
subject and then classify them using Riemannian Geometry framework for BCI. 
We compare the scores in the VR and PC conditions.

Authors: Pedro Rodrigues <pedro.rodrigues01@gmail.com>

License: BSD (3-clause)

In [2]:

# create dataset
dataset = VirtualReality()

# get the paradigm
paradigm = P300()

# loop to get scores for each subject
nsubjects = 5

df = {}
for tmax in [0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]:

	paradigm.tmax = tmax

	scores = []
	for subject in tqdm(dataset.subject_list[:nsubjects]):
		scores_subject = [subject]
		for condition in ['VR', 'PC']:

			# define the dataset instance
			if condition == 'VR':
				dataset.VR = True
				dataset.PC = False
			elif condition == 'PC':
				dataset.VR = False
				dataset.PC = True

			# get the epochs and labels
			X, labels, meta = paradigm.get_data(dataset, subjects=[subject])
			labels = LabelEncoder().fit_transform(labels)

			kf = KFold(n_splits = 6)
			repetitions = [1, 2]				
			auc = []

			blocks = np.arange(1, 12+1)
			for train_idx, test_idx in kf.split(np.arange(12)):

				# split in training and testing blocks
				X_training, labels_training, _ = get_block_repetition(X, labels, meta, blocks[train_idx], repetitions)
				X_test, labels_test, _ = get_block_repetition(X, labels, meta, blocks[test_idx], repetitions)

				# estimate the extended ERP covariance matrices with Xdawn
				dict_labels = {'Target':1, 'NonTarget':0}
				erpc = ERPCovariances(classes=[dict_labels['Target']], estimator='lwf')
				erpc.fit(X_training, labels_training)
				covs_training = erpc.transform(X_training)
				covs_test = erpc.transform(X_test)

				# get the AUC for the classification
				clf = MDM()
				clf.fit(covs_training, labels_training)
				labels_pred = clf.predict(covs_test)
				auc.append(roc_auc_score(labels_test, labels_pred))

			# stock scores
			scores_subject.append(np.mean(auc))

		scores.append(scores_subject)

	# print results
	df[tmax] = pd.DataFrame(scores, columns=['subject', 'VR', 'PC'])

filename = './results.pkl'
joblib.dump(df, filename)

The dataset class name 'VirtualReality' must be an abbreviation of its code 'Virtual-Reality-Dataset'. See moabb.datasets.base.is_abbrev for more information.
  0%|          | 0/5 [00:00<?, ?it/s]

Using default location ~/mne_data for VIRTUALREALITY...


SHA256 hash of downloaded file: 6d1db2b938e935af09ff284dc70c76c53352989f92115a8f4db79136eb031153
Use this value as the 'known_hash' argument of 'pooch.retrieve' to ensure that the file hasn't changed if it is downloaded again in the future.
  0%|          | 0/5 [00:17<?, ?it/s]


TypeError: create_info() got an unexpected keyword argument 'montage'

In [7]:
# get the paradigm
paradigm = P300()
paradigm.tmax = 1.0

# create datasets
datasets = {}
datasets['VR'] = VirtualReality()
datasets['VR'].VR = True
datasets['VR'].PC = False
datasets['PC'] = VirtualReality()
datasets['PC'].VR = False
datasets['PC'].PC = True

results = {}

for subject in VirtualReality().subject_list:

	data = {}
	data['VR'] = {}
	data['PC'] = {}

	embeddings = {}
	stats = {}
	stats['VR'] = {}
	stats['PC'] = {}

	for condition in datasets.keys():

		# get the epochs and labels
		X, y, meta = paradigm.get_data(datasets[condition], subjects=[subject])
		y = LabelEncoder().fit_transform(y)

		data[condition]['X'] = X
		data[condition]['y'] = y

		# estimate xDawn covs
		ncomps = 4
		erp = XdawnCovariances(classes=[1], estimator='lwf', nfilter=ncomps, xdawn_estimator='lwf')
		#erp = ERPCovariances(classes=[1], estimator='lwf', svd=ncomps)	
		split = train_test_split(X, y, train_size=0.50, random_state=42)
		Xtrain, Xtest, ytrain, ytest = split
		covs = erp.fit(Xtrain, ytrain).transform(Xtest)

		Mtarget = mean_riemann(covs[ytest == 1])
		Mnontarget = mean_riemann(covs[ytest == 0])
		stats[condition]['distance'] = distance_riemann(Mtarget, Mnontarget)
		stats[condition]['dispersion_target'] = np.sum([distance_riemann(covi, Mtarget)**2 for covi in covs[ytest == 1]]) / len(covs[ytest == 1])
		stats[condition]['dispersion_nontarget'] = np.sum([distance_riemann(covi, Mnontarget)**2 for covi in covs[ytest == 0]]) / len(covs[ytest == 0])
		
	print('subject', subject)
	print(stats)

	results[subject] = stats

	# covs = np.concatenate([covs, Mtarget[None,:,:], Mnontarget[None,:,:]])
	# ytest = np.concatenate([ytest, [1], [0]])
	# data[condition]['ytest'] = ytest

	# # do the embedding
	# lapl = Embedding(metric='riemann', n_components=2)
	# embeddings[condition] = lapl.fit_transform(covs)

dispersion_target_list = []
dispersion_nontarget_list = []
distance_list = []
condition_list = []
subject_list = []
for subject in results.keys():
	results_subj = results[subject]
	for condition in ['VR', 'PC']:
		subject_list.append(subject)
		condition_list.append(condition)
		dispersion_target_list.append(results_subj[condition]['dispersion_target'])
		dispersion_nontarget_list.append(results_subj[condition]['dispersion_nontarget'])
		distance_list.append(results_subj[condition]['distance'])

df = pd.DataFrame()
df['subject'] = subject_list
df['condition'] = condition_list
df['dispersion_target']	= dispersion_target_list
df['dispersion_nontarget'] = dispersion_nontarget_list
df['distance'] = distance_list

ValueError: code 'Virtual Reality dataset' must be in Camel-KebabCase; i.e. use CamelCase, and add dashes where absolutely necessary. See moabb.datasets.base.is_camel_kebab_case for more information.